In [5]:
# 继承 Module 类来构造模型
# 该类十分灵活，是一个可供自由组建的部件
# 它的子类可以是一个层，一个模型，或者是模型的一个部分

# 构造多层感知机模型

import torch
from torch import nn

In [3]:
# 定义MLP类，用于重载Module类的部分函数

class MLP(nn.Module):
    #声明带有模型参数的层，此处声明了两个全连接层，进行两次输出
    
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        
        #隐藏层
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        #输出层
        self.output = nn.Linear(256, 10)
        
        
    # 定义模型的前向计算即正向传播，根据输入返回输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [4]:
# 实例化

X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1799, -0.0725, -0.2319, -0.2010, -0.5222, -0.0069,  0.0233,  0.1987,
          0.2609, -0.1213],
        [-0.1024, -0.0318, -0.1428, -0.1129, -0.4650, -0.0859,  0.0769,  0.2309,
          0.2771, -0.0613]], grad_fn=<AddmmBackward>)

In [8]:
# Module的子类

# Sequential类
# 接收一个子模块的有序字典或者一系列子模块作为参数来逐一添加Module实例
# 模型的前向计算按实例的添加顺序逐一计算

class MySequential(nn.Module):
    from collections import OrderedDict
    
    def __init__(self, *args):
        super(MySequential, self).__init__()
        
        if len(args)==1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                #add_module方法会将module添加进self._modules(OrderedDict
                self.add_module(key, module)
                
        else:
            # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
                
    
    def forward(self, input):
        # self._modules返回一个 有序字典OrderedDict
        # 保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
            
        return input
        

In [7]:
net = MySequential(nn.Linear(784, 256),
                  nn.ReLU(),
                  nn.Linear(256, 10))
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0436, -0.1186,  0.2195,  0.0708,  0.0959,  0.1954, -0.0575, -0.0182,
         -0.2080,  0.1001],
        [ 0.0881, -0.1571,  0.3392,  0.0911,  0.2221,  0.0131,  0.0738,  0.0767,
         -0.1026, -0.0069]], grad_fn=<AddmmBackward>)

In [9]:
# ModuleList类
# 接收一个子模块的列表作为输入

net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [10]:
# ModuleDict类
# 接收一个子模块的字典作为输入

net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
